In [1]:
import numpy as np
import random
import syft as sy
import json
from syft.core.frameworks.torch import utils

In [2]:
x = sy.array([1,2,3,4])
me = sy.local_worker
hook = me.hook
me.is_client_worker = False

In [3]:
x2 = sy.FloatTensor([1,2,3,4])

In [4]:
bob = sy.VirtualWorker(hook=hook,id=2)

In [5]:
me.add_worker(bob)

In [6]:
xt = x.send(bob)
xt2 = x2.send(bob)
# xt = x.torch().send(bob)

In [7]:
# res = xt.cumsum(0)
res = xt - xt
# res

__sub__
{'command': '__sub__', 'has_self': True, 'args': (array([1, 2, 3, 4]),), 'kwargs': {}, 'self': array([1, 2, 3, 4])}


In [9]:
res = xt / xt

__truediv__
{'command': '__truediv__', 'has_self': True, 'args': (array([1, 2, 3, 4]),), 'kwargs': {}, 'self': array([1, 2, 3, 4])}


In [23]:
res.get()

array([1., 1., 1., 1.])

In [16]:
dir(np.ndarray)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_e

In [12]:
res = xt2.cumsum(0)

In [11]:
import numpy as np

In [13]:
x = np.array([1,2,3])

In [11]:
res.id

3844000594

In [9]:
bob._objects

{2224853631: array([1, 2, 3, 4]), 3844000594: array([ 1,  3,  6, 10])}

In [10]:
me._objects

{237577038: array([1, 2, 3, 4]),
 1132434330: array_ptr(None, dtype=object),
 3844000594: array([], dtype=float64),
 7623296799: array([], dtype=float64),
 62769686748: array_ptr(None, dtype=object)}

In [28]:
me.de_register(res)

In [29]:
res.owner

<syft.core.workers.virtual.VirtualWorker id:0>